<a href="https://colab.research.google.com/github/j-buss/wi-dpi-analysis/blob/development/eda/3.0_Refined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Salary and Education in Wisconsin - 3.0 Refined Data

This it the 3rd in a series of notebooks depicting the steps to acquire, store and analyze data pertaining to teachers in Wisconsin.



1.   List item
2.   List item
3.   Refined Data - Create ***all_staff_record*** tables by year



## Prep

In [1]:
!pip install --upgrade google-cloud-bigquery

     |████████████████████████████████| 133kB 4.7MB/s 
ERROR: google-cloud-translate 1.3.3 has requirement google-cloud-core<0.30dev,>=0.29.0, but you'll have google-cloud-core 1.0.1 which is incompatible.
ERROR: google-cloud-storage 1.13.2 has requirement google-cloud-core<0.30dev,>=0.29.0, but you'll have google-cloud-core 1.0.1 which is incompatible.
  Found existing installation: google-cloud-core 0.29.1
    Uninstalling google-cloud-core-0.29.1:
      Successfully uninstalled google-cloud-core-0.29.1
  Found existing installation: google-cloud-bigquery 1.8.1
    Uninstalling google-cloud-bigquery-1.8.1:
      Successfully uninstalled google-cloud-bigquery-1.8.1


### Import Libraries

In [0]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 5)
import seaborn as sns
import matplotlib.pyplot as plt

from google.cloud import bigquery

In [0]:
%matplotlib inline
plt.style.use('bmh')

### Functions

In [0]:
def create_dataset(client, project_id, dataset_name):
  
  
  dataset_id = "{}.{}".format(project_id, dataset_name)
  dataset = bigquery.Dataset(dataset_id)
  dataset.location = "US"

  dataset = client.create_dataset(dataset)
  print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

In [0]:
def convert_currency(val):
    """
    Convert the string number value to a float
     - Remove $
     - Remove commas
     - Convert to float type
    """
    new_val = val.replace(',','').replace('$', '')
    return float(new_val)

In [0]:
def prep_name(val):
  """
  Take name and make first leter capital; rest lowercase
  """
  new_val  = val.lower().title()
  return new_val

In [0]:
def create_tbl_w_sql(sql, dataset_name, table_name):
  job_config = bigquery.QueryJobConfig()
  # Set the destination table
  table_ref = bq_client.dataset(dataset_name).table(table_name)
  job_config.destination = table_ref

  # Start the query, passing in the extra configuration.
  query_job = bq_client.query(
    sql,
    # Location must match that of the dataset(s) referenced in the query
    # and of the destination table.
    location='US',
    job_config=job_config)  # API request - starts the query

  query_job.result()  # Waits for the query to finish
  print('Query results loaded to table {}'.format(table_ref.path))
  

## Processing

In [0]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

In [0]:
project_id='wi-dpi-010'
landing_dataset_name='landing'
refined_dataset_name='refined'

In [0]:
bq_client = bigquery.Client(project=project_id)

In [0]:
#Create Refined Dataset - Likely already done
##create_dataset(bq_client, project_id, refined_dataset_name)


### 2015



In [0]:
 all_staff_report_2015 = '''SELECT
  all_staff_report.id_nbr,
  TRIM(lower(all_staff_report.first_name)) as first_name,
  TRIM(lower(all_staff_report.last_name)) as last_name,
  SAFE_CAST(all_staff_report.file_number as INT64) as file_number,
  TRIM(all_staff_report.gndr) as gender,
  TRIM(all_staff_report.raceethn) as race_ethnicity_cd,
  race_ethnicity.description as race_ethnicity_desc,
  all_staff_report.birth_year,
  SAFE_CAST(all_staff_report.high_degree as INT64) as high_degree_cd,
  TRIM(highest_degree.description) as high_degree_desc,
  SAFE_CAST(REGEXP_EXTRACT(all_staff_report.year_session, r"^[0-9]{4}") AS INT64) as school_year,
  all_staff_report.cntrct_days as contract_days,
  all_staff_report.local_exp,
  all_staff_report.total_exp,
  CAST(REGEXP_REPLACE(REGEXP_REPLACE(all_staff_report.tot_salary, r"^[$]",""), r",","") AS FLOAT64) as salary,
  CAST(REGEXP_REPLACE(REGEXP_REPLACE(all_staff_report.tot_fringe, r"^[$]",""), r",","") AS FLOAT64) as benefits,
  SAFE_CAST(all_staff_report.staff_cat as INT64) as staff_category_cd,
  TRIM(staff_cat.description) as staff_category_desc,
  LPAD(CAST(all_staff_report.hire_agncy_typ AS STRING), 2, "0") as hire_agency_type_cd,
  hire_agency_type.description as hire_agency_type_desc,
  all_staff_report.hire_agncy_cd as hire_agency_cd,
  LPAD(CAST(all_staff_report.work_agncy_typ AS STRING), 2, "0") as work_agency_type_cd,
  work_agency_type.description as work_agency_type_desc,
  all_staff_report.work_agncy_cd as work_agency_cd,
  TRIM(all_staff_report.school_cd) as school_cd,
  position.position_type as position_type_cd,
  pos_type.description as position_type_desc,
  all_staff_report.position_cd,
  position.position_description,
  all_staff_report.assgn_area_cd as assignment_area_cd,
  assignment_area.assignment_area_description,
  TRIM(all_staff_report.low_grd) as low_grade,
  TRIM(all_staff_report.high_grd) as high_grade,
  all_staff_report.lg_sort_cd,
  all_staff_report.hg_sort_cd,
  all_staff_report.bilingual,
  all_staff_report.assgn_fte,
  TRIM(all_staff_report.work_location_name) as work_location_name,
  TRIM(all_staff_report.school_name) as school_name,
  TRIM(all_staff_report.grd_level) as grade_level,
  SAFE_CAST(TRIM(all_staff_report.cesa_number) as INT64) as cesa_num,
  all_staff_report.cnty_nbr as county_number,
  TRIM(all_staff_report.cnty_name) as county_name,
  TRIM(all_staff_report.school_mailing_address1) as school_mailing_address1,
  TRIM(all_staff_report.school_mailing_address2) as school_mailing_address2,
  TRIM(all_staff_report.mail_city) as mail_city,
  TRIM(all_staff_report.mail_st) as mail_st,
  TRIM(all_staff_report.mail_zip_cd) as mail_zip_cd,
  TRIM(all_staff_report.school_shipping_address1) as school_shipping_address1,
  TRIM(all_staff_report.school_shipping_address2) as school_shipping_address2,
  TRIM(all_staff_report.mail_city) as ship_city,
  TRIM(all_staff_report.mail_st) as ship_st,
  TRIM(all_staff_report.mail_zip_cd) as ship_zip_cd,
  TRIM(all_staff_report.phone) as phone,
  TRIM(all_staff_report.admin_name) as admin_name,
  TRIM(all_staff_report.former_last_nm) as former_last_name,
  TRIM(all_staff_report.lt_sub) as long_term_sub,
  TRIM(all_staff_report.sub_cntrctd) as sub_contracted
FROM
  `wi-dpi-010.landing.2015_all_staff_report` all_staff_report 
  LEFT JOIN `wi-dpi-010.landing.2015_positions` position 
   ON all_staff_report.position_cd = position.code
  LEFT JOIN `wi-dpi-010.landing.2015_assignment_area` assignment_area
   ON all_staff_report.assgn_area_cd = CAST(assignment_area.code as INT64)
  LEFT JOIN `wi-dpi-010.landing.2015_highest_educational_degree` highest_degree
   ON SAFE_CAST(all_staff_report.high_degree as INT64) = highest_degree.code
  LEFT JOIN `wi-dpi-010.landing.2015_staff_category` staff_cat
   ON SAFE_CAST(all_staff_report.staff_cat as INT64) = staff_cat.code
  LEFT JOIN `wi-dpi-010.landing.2015_position_type` pos_type
   ON position.position_type = pos_type.code
  LEFT JOIN `wi-dpi-010.landing.2015_agency_type` hire_agency_type
   ON LPAD(CAST(all_staff_report.hire_agncy_typ AS STRING), 2, "0") = hire_agency_type.code
  LEFT JOIN `wi-dpi-010.landing.2015_agency_type` work_agency_type
   ON LPAD(CAST(all_staff_report.work_agncy_typ AS STRING), 2, "0") = work_agency_type.code
  LEFT JOIN `wi-dpi-010.landing.2015_race` race_ethnicity
   ON TRIM(all_staff_report.raceethn) = race_ethnicity.code
'''

In [0]:
create_tbl_w_sql(all_staff_report_2015 , refined_dataset_name, '2015_all_staff_report')

Query results loaded to table /projects/wi-dpi-010/datasets/refined/tables/2015_all_staff_report


### 2016

In [0]:
update_2016_agency_type='''
UPDATE
  `wi-dpi-010.landing.2016_agency_type` 
SET
  code = LPAD(CAST(code AS STRING), 2, "0")
WHERE
  LENGTH(code) < 2
'''

In [0]:
query_job=bq_client.query(update_2016_agency_type)
results = query_job.result()

In [0]:
all_staff_report_2016='''
SELECT
  all_staff_report.id_nbr,
  TRIM(lower(all_staff_report.first_name)) as first_name,
  TRIM(lower(all_staff_report.last_name)) as last_name,
  SAFE_CAST(all_staff_report.file_number as INT64) as file_number,
  TRIM(all_staff_report.gndr) as gender,
  TRIM(all_staff_report.raceethn) as race_ethnicity_cd,
  race_ethnicity.description as race_ethnicity_desc,  
  all_staff_report.birth_year,
  SAFE_CAST(all_staff_report.high_degree as INT64) as high_degree_cd,
  TRIM(highest_degree.description) as high_degree_desc,
  SAFE_CAST(REGEXP_EXTRACT(all_staff_report.year_session, r"^[0-9]{4}") AS INT64) as school_year,
  all_staff_report.cntrct_days as contract_days,
  all_staff_report.local_exp,
  all_staff_report.total_exp,
  CAST(REGEXP_REPLACE(REGEXP_REPLACE(all_staff_report.tot_salary, r"^[$]",""), r",","") AS FLOAT64) as salary,
  CAST(REGEXP_REPLACE(REGEXP_REPLACE(all_staff_report.tot_fringe, r"^[$]",""), r",","") AS FLOAT64) as benefits,
  SAFE_CAST(all_staff_report.staff_cat as INT64) as staff_category_cd,
  TRIM(staff_cat.description) as staff_category_desc,
  LPAD(CAST(all_staff_report.hire_agncy_typ AS STRING), 2, "0") as hire_agency_type_cd,
  hire_agency_type.description as hire_agency_type_desc,
  all_staff_report.hire_agncy_cd as hire_agency_cd,
  LPAD(CAST(all_staff_report.work_agncy_typ AS STRING), 2, "0") as work_agency_type_cd,
  work_agency_type.description as work_agency_type_desc,
  all_staff_report.work_agncy_cd as work_agency_cd,
  TRIM(all_staff_report.school_cd) as school_cd,
  position.position_type as position_type_cd,
  pos_type.description as position_type_desc,
  all_staff_report.position_cd,
  position.position_description,
  all_staff_report.assgn_area_cd as assignment_area_cd,
  assignment_area.assignment_area_description,
  TRIM(all_staff_report.low_grd) as low_grade,
  TRIM(all_staff_report.high_grd) as high_grade,
  all_staff_report.lg_sort_cd,
  all_staff_report.hg_sort_cd,
  all_staff_report.bilingual,
  all_staff_report.assgn_fte,
  TRIM(all_staff_report.work_location_name) as work_location_name,
  TRIM(all_staff_report.school_name) as school_name,
  TRIM(all_staff_report.grd_level) as grade_level,
  SAFE_CAST(TRIM(all_staff_report.cesa_number) as INT64) as cesa_num,
  all_staff_report.cnty_nbr as county_number,
  TRIM(all_staff_report.cnty_name) as county_name,
  TRIM(all_staff_report.school_mailing_address1) as school_mailing_address1,
  TRIM(all_staff_report.school_mailing_address2) as school_mailing_address2,
  TRIM(all_staff_report.mail_city) as mail_city,
  TRIM(all_staff_report.mail_st) as mail_st,
  TRIM(all_staff_report.mail_zip_cd) as mail_zip_cd,
  TRIM(all_staff_report.school_shipping_address1) as school_shipping_address1,
  TRIM(all_staff_report.school_shipping_address2) as school_shipping_address2,
  TRIM(all_staff_report.mail_city) as ship_city,
  TRIM(all_staff_report.mail_st) as ship_st,
  TRIM(all_staff_report.mail_zip_cd) as ship_zip_cd,
  TRIM(all_staff_report.phone) as phone,
  TRIM(all_staff_report.admin_name) as admin_name,
  TRIM(all_staff_report.former_last_nm) as former_last_name,
  TRIM(all_staff_report.lt_sub) as long_term_sub,
  TRIM(all_staff_report.sub_cntrctd) as sub_contracted
FROM
  `wi-dpi-010.landing.2016_all_staff_report` all_staff_report 
  LEFT JOIN `wi-dpi-010.landing.2016_positions` position 
   ON all_staff_report.position_cd = position.code
  LEFT JOIN `wi-dpi-010.landing.2016_assignment_area` assignment_area
   ON all_staff_report.assgn_area_cd = CAST(assignment_area.code as INT64)
  LEFT JOIN `wi-dpi-010.landing.2016_highest_educational_degree` highest_degree
   ON SAFE_CAST(all_staff_report.high_degree as INT64) = highest_degree.code
  LEFT JOIN `wi-dpi-010.landing.2016_staff_category` staff_cat
   ON SAFE_CAST(all_staff_report.staff_cat as INT64) = staff_cat.code
  LEFT JOIN `wi-dpi-010.landing.2016_position_type` pos_type
   ON position.position_type = pos_type.code
  LEFT JOIN `wi-dpi-010.landing.2016_agency_type` hire_agency_type
   ON all_staff_report.hire_agncy_typ = hire_agency_type.code
  LEFT JOIN `wi-dpi-010.landing.2016_agency_type` work_agency_type
   ON all_staff_report.work_agncy_typ = work_agency_type.code
  LEFT JOIN `wi-dpi-010.landing.2016_race` race_ethnicity
   ON TRIM(all_staff_report.raceethn) = race_ethnicity.code
'''

In [0]:
create_tbl_w_sql(all_staff_report_2016, refined_dataset_name, '2016_all_staff_report')

Query results loaded to table /projects/wi-dpi-010/datasets/refined/tables/2016_all_staff_report


### 2017

In [0]:
all_staff_report_2017='''
SELECT
  all_staff_report.research_id,
  all_staff_report.school_year as year_session,
  TRIM(lower(all_staff_report.first_name)) as first_name,
  TRIM(lower(all_staff_report.last_name)) as last_name,
  SAFE_CAST(all_staff_report.entity_id as INT64) as entity_id,
  TRIM(all_staff_report.gender) as gender,
  TRIM(SPLIT(all_staff_report.raceethnicity, " - ")[safe_offset(0)]) as race_ethnicity_cd,
  TRIM(SPLIT(all_staff_report.raceethnicity, " - ")[safe_offset(1)]) as race_ethnicity_desc,
  all_staff_report.birth_year,
  TRIM(SPLIT(all_staff_report.contract_hire_agency," - ")[safe_offset(0)]) as contract_hire_agency_cd,
  TRIM(SPLIT(all_staff_report.contract_hire_agency," - ")[safe_offset(1)]) as contract_hire_agency_desc,
  TRIM(SPLIT(all_staff_report.contract_high_degree, " - ")[safe_offset(0)]) as contract_high_degree_cd,
  TRIM(SPLIT(all_staff_report.contract_high_degree, " - ")[safe_offset(1)]) as contract_high_degree_desc,
  SAFE_CAST(all_staff_report.contract_days as INT64) as contract_days,
  all_staff_report.contract_local_experience,
  all_staff_report.contract_total_experience,
  CAST(REGEXP_REPLACE(REGEXP_REPLACE(all_staff_report.total_salary, r"^[$]",""), r",","") AS FLOAT64) as salary,
  CAST(REGEXP_REPLACE(REGEXP_REPLACE(all_staff_report.total_fringe, r"^[$]",""), r",","") AS FLOAT64) as benefits,
  TRIM(SPLIT(all_staff_report.assignment_area, " - ")[safe_offset(0)]) as assignment_area_cd,
  TRIM(SPLIT(all_staff_report.assignment_area, " - ")[safe_offset(1)]) as assignment_area_desc,
  TRIM(SPLIT(all_staff_report.assignment_staff_category, " - ")[safe_offset(0)]) as assignment_staff_category_cd,
  TRIM(SPLIT(all_staff_report.assignment_staff_category, " - ")[safe_offset(1)]) as assignment_staff_category_desc,
  all_staff_report.position_classification,
  TRIM(SPLIT(all_staff_report.hire_agency, " - ")[safe_offset(0)]) as hire_agency_cd,
  TRIM(SPLIT(all_staff_report.hire_agency, " - ")[safe_offset(1)]) as hire_agency_desc,
  TRIM(SPLIT(all_staff_report.assignment_hire_agency_type, " - ")[safe_offset(0)]) as assignment_hire_agency_type_cd,
  TRIM(SPLIT(all_staff_report.assignment_hire_agency_type, " - ")[safe_offset(1)]) as assignment_hire_agency_type_desc,
  TRIM(SPLIT(all_staff_report.assignment_work_agency, " - ")[safe_offset(0)]) as assignment_work_agency_cd,
  TRIM(SPLIT(all_staff_report.assignment_work_agency, " - ")[safe_offset(1)]) as assignment_work_agency_desc,
  TRIM(SPLIT(all_staff_report.work_agency_type, " - ")[safe_offset(0)]) as work_agency_type_cd,
  TRIM(SPLIT(all_staff_report.work_agency_type, " - ")[safe_offset(1)]) as work_agency_type_desc,
  CASE
    WHEN REGEXP_CONTAINS(all_staff_report.assignment_work_school," - ") THEN TRIM(SPLIT(all_staff_report.assignment_work_school, " - ")[safe_OFFSET(0)])
  ELSE
    ""
  END AS assignment_work_school_cd,
  CASE
    WHEN REGEXP_CONTAINS(all_staff_report.assignment_work_school," - ") THEN TRIM(SPLIT(all_staff_report.assignment_work_school, " - ")[safe_OFFSET(1)])
  ELSE
    TRIM(all_staff_report.assignment_work_school)
  END AS assignment_work_school_desc,
  SAFE_CAST(all_staff_report.assignment_work_cesa_number as INT64) as cesa_num,
  TRIM(SPLIT(all_staff_report.assignment_work_county, " - ")[safe_offset(0)]) as assignment_work_county_cd,
  TRIM(SPLIT(all_staff_report.assignment_work_county, " - ")[safe_offset(1)]) as assignment_work_county_name,
  TRIM(SPLIT(all_staff_report.assignment_work_school_level, " - ")[safe_offset(0)]) as assignment_work_school_level_cd,
  TRIM(SPLIT(all_staff_report.assignment_work_school_level, " - ")[safe_offset(1)]) as assignment_work_school_level_desc,
  TRIM(SPLIT(all_staff_report.assignment_position, " - ")[safe_offset(0)]) as assignment_position_cd,
  TRIM(SPLIT(all_staff_report.assignment_position, " - ")[safe_offset(1)]) as assignment_position_desc,
  all_staff_report.assignment_fte,
  TRIM(all_staff_report.assignment_grades_served) as assignment_grades_served,
  TRIM(all_staff_report.assignment_long_term_substitute) as assignment_long_term_substitute,
  TRIM(all_staff_report.assignment_bilingual_program) as assignment_bilingual_program,
  TRIM(all_staff_report.assignment_alternative_program) as assignment_alternative_program,
  TRIM(all_staff_report.assignment_subcontracted) as assignment_subcontracted,
  TRIM(all_staff_report.assignment_requires_dpi_license) as assignment_requires_dpi_license,
  TRIM(all_staff_report.school_mailing_street_address) as school_mailing_street_address,
  SAFE_CAST(all_staff_report.district_mailing_po_box as INT64) as school_mailing_po_box,
  TRIM(all_staff_report.school_mailing_city) as school_mailing_city,
  TRIM(all_staff_report.school_mailing_state) as school_mailing_state,
  SAFE_CAST(all_staff_report.school_mailing_zip_code as String) as school_mailing_zip_code,
  TRIM(all_staff_report.district_mailing_street_address) as district_mailing_street_address,
  SAFE_CAST(all_staff_report.district_mailing_po_box as INT64) as district_mailing_po_box,
  TRIM(all_staff_report.district_mailing_city) as district_mailing_city,
  TRIM(all_staff_report.district_mailing_state) as district_mailing_state,
  SAFE_CAST(all_staff_report.district_mailing_zip_code as String) as district_mailing_zip_code
FROM
  `wi-dpi-010.landing.2017_all_staff_report` all_staff_report
'''

In [0]:
create_tbl_w_sql(all_staff_report_2017, refined_dataset_name, '2017_all_staff_report')

Query results loaded to table /projects/wi-dpi-010/datasets/refined/tables/2017_all_staff_report


## Consolidate Yearly Tables

In [0]:
consolidate_select='''
'''